<a href="https://colab.research.google.com/github/kelvinfoo123/Natural-Language-Processing/blob/main/Fake_News_Classification_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this project, I experiment using LSTM to classify fake news by looking at the headline. 



In [ ]:
import pandas as pd 
import numpy as np 

In [ ]:
news = pd.read_csv("data.csv")
news.head()

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1


In [ ]:
news.isnull().sum() # No null values in the headline. 

URLs         0
Headline     0
Body        21
Label        0
dtype: int64

In [ ]:
X = news['Headline']
y = news['Label']

print(X.shape)
print(y.shape)

(4009,)
(4009,)


**Text Preprocessing**

In [ ]:
import nltk 
import re 
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')
stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
corpus = []

for i in range(0, len(X)): 
  review = re.sub('[^a-zA-Z]', ' ', X[i]) # For every headline, if the character is not a word, replace with ' '. 
  review = review.lower() # Make all words lower case. The same word in different cases are treated as different words. 
  review = review.split()
  review = [word for word in review if not word in stopwords.words('english')] # Remove stopwords 
  review = [stemmer.stem(word) for word in review] # Stemming 

  review = ' '.join(review)
  corpus.append(review)

In [ ]:
for i in range(1,11): 
  print(corpus[i])

linklat war veteran comedi speak modern america say star
trump fight corker jeopard legisl agenda
egypt cheiron win tie pemex mexican onshor oil field
jason aldean open snl vega tribut
jetnat fanduel leagu week
kansa tri tax plan similar trump fail
india rbi chief growth import cost inflat newspap
epa chief sign rule clean power plan exit tuesday
talk sale air berlin plane easyjet risk collaps report
u presid donald trump quietli sign law allow warrantless search part va dc md


**One Hot Representation**

One-hot encodes a text into a list of word indexes of size n, where n is the size of the vocab. 

Reason for encoding is to apply embedding as embedding requires input data to be integer encoded. 

In [ ]:
import tensorflow as tf 
from tensorflow.keras.preprocessing.text import one_hot

In [ ]:
# Vocabulary size 
voc_size = 10000

In [ ]:
onehot_repre = [one_hot(words, voc_size) for words in corpus]

for i in range(11): 
  print(onehot_repre[i])

[5158, 6484, 6823, 1668, 6327, 7425, 7412]
[1926, 9453, 7126, 2586, 650, 4102, 1761, 625, 1150]
[7412, 8097, 1668, 7841, 2446, 3487]
[9447, 1487, 9641, 1285, 7460, 1575, 3554, 8378, 9970]
[2313, 7904, 2066, 7133, 7700, 6097]
[4913, 2247, 4938, 7649]
[7637, 4985, 4370, 8277, 2331, 7412, 2751]
[4202, 8294, 7452, 2864, 3376, 9890, 4293, 7879]
[1087, 7452, 6186, 6904, 4795, 6410, 8277, 6735, 6574]
[3723, 9017, 2443, 4556, 3568, 2867, 5526, 8141, 593]
[5026, 8720, 7425, 7412, 8876, 6186, 8525, 4206, 3298, 7037, 6436, 7052, 3169, 4887]


**Padding Sequences**

LSTM require that input sequences have the same length. We make all input sequences have the same length by padding 0 to the input until they have the stated length. 

In [ ]:
# Library for padding 

from tensorflow.keras.preprocessing.sequence import pad_sequences 

In [ ]:
length = 30 # Require that all inputs have length 30 

padded = pad_sequences(onehot_repre, padding = 'pre', maxlen = length) # Padding = 'pre' means pad in front of the input. 

for i in range(11): 
  print(padded[i])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0 5158 6484 6823 1668 6327
 7425 7412]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0 1926 9453 7126 2586  650 4102 1761
  625 1150]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0 7412 8097 1668 7841
 2446 3487]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0 9447 1487 9641 1285 7460 1575 3554
 8378 9970]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0 2313 7904 2066 7133
 7700 6097]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0 4913 2247
 4938 7649]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0

**Building Model**

Before running LSTM, we need an embedding layer. Compared to traditional bag of words model where representation vectors are sparse due to large vocabularies, words are represented by dense vectors in an embedding. 

The embedding layer requires that the input data be integer encoded, so that each word is represented by a unique integer. 

The embedding layer has 3 arguments: 


*   input_dim: Size of vocab
*   output_dim: Size of vector space in which word will be embedded. 
*   input_length: Length of input sentence 


The embedding layer will be trained as part of the neural network. The output is a 2D vector with one embedding for each word in the input. 


In [ ]:
from tensorflow.keras import regularizers 
from tensorflow.keras.layers import Embedding 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import LSTM 
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
padded_final = np.array(padded)
y_final = np.array(y)

from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(padded_final, y_final, test_size = 0.3, random_state = 42)

In [ ]:
vector_size = 40 # Size of vector space in which word will be embedded.

model = Sequential()
model.add(Embedding(voc_size, vector_size, input_length = length))
model.add(LSTM(100)) # One LSTM layer with 100 neurons 
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
44/44 [==============================] - 5s 60ms/step - loss: 0.6771 - accuracy: 0.5638 - val_loss: 0.6025 - val_accuracy: 0.7240
Epoch 2/10
44/44 [==============================] - 2s 45ms/step - loss: 0.4269 - accuracy: 0.8532 - val_loss: 0.4282 - val_accuracy: 0.7955
Epoch 3/10
44/44 [==============================] - 2s 45ms/step - loss: 0.2182 - accuracy: 0.9227 - val_loss: 0.4030 - val_accuracy: 0.8155
Epoch 4/10
44/44 [==============================] - 3s 76ms/step - loss: 0.1179 - accuracy: 0.9629 - val_loss: 0.4571 - val_accuracy: 0.8288
Epoch 5/10
44/44 [==============================] - 2s 49ms/step - loss: 0.0655 - accuracy: 0.9804 - val_loss: 0.5188 - val_accuracy: 0.8254
Epoch 6/10
44/44 [==============================] - 3s 58ms/step - loss: 0.0382 - accuracy: 0.9882 - val_loss: 0.5901 - val_accuracy: 0.8279
Epoch 7/10
44/44 [==============================] - 3s 68ms/step - loss: 0.0280 - accuracy: 0.9929 - val_loss: 0.5163 - val_accuracy: 0.8180
Epoch 8/10
44

The training accuracy is much higher than the test accuracy. This might be a sign of overfitting. We implement dropout or regularization. 

In [ ]:
model = Sequential()
model.add(Embedding(voc_size, vector_size, input_length = length))
model.add(Dropout(0.5))
model.add(LSTM(100))
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'sigmoid', kernel_regularizer = regularizers.l2(0.01)))

In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
44/44 [==============================] - 5s 63ms/step - loss: 0.7054 - accuracy: 0.5314 - val_loss: 0.6941 - val_accuracy: 0.6916
Epoch 2/10
44/44 [==============================] - 2s 50ms/step - loss: 0.6157 - accuracy: 0.7324 - val_loss: 0.5090 - val_accuracy: 0.7714
Epoch 3/10
44/44 [==============================] - 2s 50ms/step - loss: 0.3859 - accuracy: 0.8614 - val_loss: 0.4559 - val_accuracy: 0.7972
Epoch 4/10
44/44 [==============================] - 2s 49ms/step - loss: 0.2574 - accuracy: 0.9141 - val_loss: 0.4441 - val_accuracy: 0.8105
Epoch 5/10
44/44 [==============================] - 2s 49ms/step - loss: 0.1908 - accuracy: 0.9408 - val_loss: 0.4303 - val_accuracy: 0.8155
Epoch 6/10
44/44 [==============================] - 2s 47ms/step - loss: 0.1415 - accuracy: 0.9629 - val_loss: 0.4467 - val_accuracy: 0.8171
Epoch 7/10
44/44 [==============================] - 2s 48ms/step - loss: 0.1233 - accuracy: 0.9701 - val_loss: 0.5175 - val_accuracy: 0.8213
Epoch 8/10
44

**Bidirectional LSTM**

In [ ]:
from tensorflow.keras.layers import Bidirectional

In [ ]:
vector_size = 40 

model = Sequential()
model.add(Embedding(voc_size, vector_size, input_length = length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
44/44 [==============================] - 6s 74ms/step - loss: 0.6861 - accuracy: 0.5859 - val_loss: 0.6513 - val_accuracy: 0.6185
Epoch 2/10
44/44 [==============================] - 2s 54ms/step - loss: 0.4859 - accuracy: 0.8001 - val_loss: 0.4184 - val_accuracy: 0.8030
Epoch 3/10
44/44 [==============================] - 2s 53ms/step - loss: 0.2015 - accuracy: 0.9241 - val_loss: 0.4790 - val_accuracy: 0.8180
Epoch 4/10
44/44 [==============================] - 2s 52ms/step - loss: 0.1043 - accuracy: 0.9661 - val_loss: 0.5114 - val_accuracy: 0.8271
Epoch 5/10
44/44 [==============================] - 2s 53ms/step - loss: 0.0699 - accuracy: 0.9779 - val_loss: 0.5646 - val_accuracy: 0.8379
Epoch 6/10
44/44 [==============================] - 2s 52ms/step - loss: 0.0399 - accuracy: 0.9904 - val_loss: 0.6368 - val_accuracy: 0.8437
Epoch 7/10
44/44 [==============================] - 2s 54ms/step - loss: 0.0259 - accuracy: 0.9936 - val_loss: 0.6957 - val_accuracy: 0.8437
Epoch 8/10
44